In [1]:
# from SpinHamiltonian import ZeemanSpinHamiltonian, HyperfineSpinHamiltonian, ElectronicSpinHamiltonian

import numpy as np
import pandas as pd
import qutip as qt
import math

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['font.size'] = 22

In [2]:
def get_field_components(filename, vb, header=7,):
    data =  pd.read_csv(filename, header=header)
    names = ['cut_length_um']
    for v in vb:
        names.append(str(v) + 'V')
    data.columns = names 
    return data

vb = np.arange(0,52,1)
Ex = get_field_components('data/Ex.txt',vb)

Ex_z0um = [Ex[Ex.columns[i]][0] for i in range(len(Ex.columns)-1)]

In [3]:
class SpinHamiltonian:
    def __init__(self):
        self.sx = qt.jmat(1,'x')
        self.sy = qt.jmat(1,'y')
        self.sz = qt.jmat(1,'z')

        self.ge = 28e09 # Gyromagnetic ratio of NV center
        self.D = 2.87e09 # Zero field splitting

        self.zero_field = self.D*((self.sz*self.sz)-(2/3)*qt.qeye(3))
        
        self.ms0 = 0 # Energy of ms=0 state initialized to 0
        self.dpar = 0.03
        self.dperp = 0.17

    def zeeman_interaction(self,B,theta,z):
        if z:
            Bx = 0
            By = 0
            Bz = B
        else:
            Bx = B * math.cos(theta) # calculating B from its magnitude, polar angle
            By = B * math.sin(theta)
            Bz = 0
        Hzee = self.ge*(Bz*self.sz + Bx*self.sx + By*self.sy)
        return Hzee
    
    def electronic_interaction(self,E, theta, z):# z is a flag to distinguish between parallel and perpendicular
        Ex = E*math.cos(theta) # calculating E from its magnitude, polar angle.
        Ey = E*math.sin(theta)
        Ez=0
        if z :
            Ex =0
            Ey =0
            Ez = E
            
        Hes = (self.dpar * (Ez*self.sz**2) -  
               self.dperp * (Ex*(self.sx**2-self.sy**2)) +  
               self.dpar * (Ey*(self.sx*self.sy + self.sy*self.sx)) )
        return Hes

    def _get_transition_frequencies(self, B, Btheta, BzFlag, E, Etheta, EzFlag):
        H = self.zero_field + self.zeeman_interaction(B,Btheta,BzFlag) + self.electronic_interaction(E, Etheta, EzFlag)
        estates = H.eigenstates()
        egvals = estates[0]
        
        if (BzFlag or EzFlag):
            z = 1
        else:
            z = 0
        
        if(B == 0): self.ms0 = egvals[0] 
        f1 = egvals[2] - self.ms0 if(z) else egvals[2]-egvals[0] # to distinguish parallel and perpendiculr energies as qutip sorts them
        f0 = abs(egvals[1] + egvals[0] - (2*self.ms0)) if(z) else egvals[1]-egvals[0] # absolute value of frequency

        return np.array([f1,f0])
    
    def transition_frequencies(self, B, Btheta, BzFlag, E, Etheta, EzFlag):
        """ Need to generalize this for all  fields """
        ham = np.vectorize(self._get_transition_frequencies, otypes=[np.ndarray])
        # ham = np.vectorize(self.zeeman.transition_frequencies, otypes=[np.ndarray])
        trans_freqs = np.array(ham(B, Btheta, BzFlag, E, Etheta, EzFlag))
        return np.array(trans_freqs.tolist())

In [64]:
class HyperfineSpinHamiltonian(SpinHamiltonian):
    def __init__(self):
        SpinHamiltonian.__init__(self)
        self.gc = 10.705e6                  # gyromagnetic ratio of C-13 nucleus in Hz/T for hyperfine interaction

        self.Axx = 189.3e6
        self.Ayy = 128.4e6
        self.Azz = 128.9e6
        self.Axz = 24.1e6                   # Hyperfine Tensor components in NV frame of reference. Taken from reference 2.
        self.Ix = qt.jmat(.5,'x')
        self.Iy = qt.jmat(.5,'y')
        self.Iz = qt.jmat(.5,'z')            # Spin 1/2 operators for C-13 nucleus

        self.comp1 = self.Axx*qt.tensor(self.sx,self.Ix)
        self.comp2 = self.Ayy*qt.tensor(self.sy,self.Iy)
        self.comp3 = self.Azz*qt.tensor(self.sz,self.Iz)
        self.comp4 = self.Axz*(qt.tensor(self.sx,self.Iz)+qt.tensor(self.sz,self.Ix))
        self.Hhf = self.comp1 + self.comp2 + self.comp3 + self.comp4

        self.zero_field = self.D*(qt.tensor(self.sz*self.sz,qt.qeye(2))-(2/3)*qt.tensor(qt.qeye(3),qt.qeye(2)))
        
    def hyperfine_interaction(self,Bz):
        H = self.gc*Bz*qt.tensor(qt.qeye(3),self.Iz) + self.Hhf  
        return H.eigenstates()

    def interaction_with_field(self, Bz, E, Etheta, EzFlag):
        """ CHANGE THIS TO CALL OTHER CLASSES """
#         import pdb; pdb.set_trace()
#         H = (self.zero_field + 
#              self.ge*Bz*qt.tensor(self.sz,qt.qeye(3)) + 
#              self.hyperfine_interaction(Bz) + 
#              self.electronic_interaction(E, Etheta, EzFlag))
    
        H = (self.D*(qt.tensor(self.sz*self.sz,qt.qeye(2))-(2/3)*qt.tensor(qt.qeye(3),qt.qeye(2))) + 
             self.ge*Bz*qt.tensor(self.sz,qt.qeye(2)) + self.gc*Bz*qt.tensor(qt.qeye(3),self.Iz) + 
             qt.tensor(self.electronic_interaction(E, Etheta, EzFlag),qt.qeye(2)) +
             self.Hhf) 

#         print(np.size(self.zero_field)) 
#         print(np.size(self.ge*Bz*qt.tensor(self.sz,qt.qeye(3))))
#         print(np.size(self.hyperfine_interaction(Bz)))
#         print(np.size(self.electronic_interaction(E, Etheta, EzFlag)))
        return H[0]

    def transition_freqs(self, Bz, E, Etheta, EzFlag):
        egvals = self.interaction_with_field(Bz, E, Etheta, EzFlag)[0]
#         print(egvals)
        ms0 = (egvals[0] + egvals[1] + egvals[2])/3		# energy of 0 level averaged for simpler graph
        return np.array([egvals[2]-ms0, egvals[3]-ms0, egvals[4]-ms0, egvals[5]-ms0])

In [65]:
ham = HyperfineSpinHamiltonian()

In [68]:
ham.transition_freqs(1,1,0,0)

array([-9.67382596e+09+0.j, -9.66081520e+09+0.j, -9.68234660e+09+0.j,
       -9.68234660e+09+0.j])

In [8]:
class HyperfineSpinHalf():
    def __init__(self):
        self.sx = qt.jmat(1,'x')
        self.sy = qt.jmat(1,'y')
        self.sz = qt.jmat(1,'z')            # jmat is higher order spin operator. 1 in this case.
        self.ge = 28e9                      # gyromagnetic ratio of electron in Hz/T
        self.gc = 10.705e6                  # gyromagnetic ratio of C-13 nucleus in Hz/T for hyperfine interaction
        self.D = 2870.2e6                   # zero field splitting frequency in Hz

        self.Axx = 189.3e6
        self.Ayy = 128.4e6
        self.Azz = 128.9e6
        self.Axz = 24.1e6                   # Hyperfine Tensor components in NV frame of reference. Taken from reference 2.
        self.Ix = qt.jmat(0.5,'x')
        self.Iy = qt.jmat(0.5,'y')
        self.Iz = qt.jmat(0.5,'z')          # Spin 1/2 operators for C-13 nucleus

        self.comp1 = self.Axx*qt.tensor(self.sx,self.Ix)
        self.comp2 = self.Ayy*qt.tensor(self.sy,self.Iy)
        self.comp3 = self.Azz*qt.tensor(self.sz,self.Iz)
        self.comp4 = self.Axz*(qt.tensor(self.sx,self.Iz)+qt.tensor(self.sz,self.Ix))
        self.Hhf = self.comp1 + self.comp2 + self.comp3 + self.comp4
        
        self.zero_field = self.D*(qt.tensor(self.sz*self.sz,qt.qeye(2))-(2/3)*qt.tensor(qt.qeye(3),qt.qeye(2)))
        print(np.size(self.zero_field))

    def transitionFreqs(self,Bz):
        H = self.D*(qt.tensor(self.sz*self.sz,qt.qeye(2))-(2/3)*qt.tensor(qt.qeye(3),qt.qeye(2))) + self.ge*Bz*qt.tensor(self.sz,qt.qeye(2)) + self.gc*Bz*qt.tensor(qt.qeye(3),self.Iz) + self.Hhf 
        print(np.size(H))
        
        egvals = H.eigenstates()[0]

        ms0 = (egvals[0] + egvals[1])/2		# zero level frequency averaged for simpler B vs F graph
        return np.array([egvals[2]-ms0, egvals[3]-ms0, egvals[4]-ms0, egvals[5]-ms0])

    def eigenvalues(self,Bz):
        H = self.D*(qt.tensor(self.sz*self.sz,qt.qeye(2))-(2/3)*qt.tensor(qt.qeye(3),qt.qeye(2))) + self.ge*Bz*qt.tensor(self.sz,qt.qeye(2)) + self.gc*Bz*qt.tensor(qt.qeye(3),self.Iz) + self.Hhf 
        
        return H.eigenstates()[0]

In [9]:
ham = HyperfineSpinHalf()
# ham.transitionFreqs(1)

36
